In [2]:
import torch
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [1]:
from typing import Tuple
from io import BytesIO
import os
import argparse
import re
import fitz

def search_for_text(lines, search_str):
    """
    Search for the search string within the document lines
    """
    for line in lines:
        # Find all matches within one line
        results = re.findall(search_str, line, re.IGNORECASE)
        # In case multiple matches within one line
        for result in results:
            yield result


In [2]:
file_path = "demo.pdf"
search_str = "sample"

# Open the PDF file
pdf_document = fitz.open(file_path)

# Iterate over all pages
for page_num in range(pdf_document.page_count):
    # Get the page
    page = pdf_document.load_page(page_num)
    # Get the text
    text = page.get_text()
    # Split the text into lines
    lines = text.split("\n")
    # Search for the search string within the lines
    for result in search_for_text(lines, search_str):
        print(f"Found '{result}' on page {page_num}")
        print(f"Page text: {text}")


Found 'sample' on page 1
Page text: 1.
Text File
a.
PII identification (https://github.com/i-dot-ai/redactomatic)
b.
Advanced - Replace with scenthetic information / scramble text
c.
Ensure no input data is stored or retrievable after processing.
d.
https://www.youtube.com/watch?v=nJKNLl9W-2E
2.
Images / VIDEOS
a.
OCR for Text: Extract text from images and apply the same redaction as the text
modality.
b.
For documents or images containing sensitive objects (e.g., license plates,
sensitive labels), detect and redact these areas.
c.
Grounding Dino for custom
d.
BLURING / BLACK BOX / PIXELATION
e.
(replacement with synthetic objects)
3.
PDF
a.
https://stackoverflow.com/questions/75387339/how-can-i-edit-modify-replace-text
-in-an-existing-pdf-file
b.
Same as text
c.
Resources -
https://huggingface.co/docs/transformers/tasks/token_classification
https://huggingface.co/datasets/ai4privacy/pii-masking-300k
https://medium.com/@bisinet/obfuscating-python-code-f6537e5c51a3
https://microsoft.git

In [6]:
import pymupdf

# Open the PDF document
doc = pymupdf.open('demo.pdf')

# Iterate over each page of the document
for page in doc:
    # Find all instances of "Jane Doe" on the current page
    instances = page.search_for("sample")

    # Redact each instance of "Jane Doe" on the current page
    for inst in instances:
        page.add_redact_annot(inst , fill=(0,0,0))

    # Apply the redactions to the current page
    page.apply_redactions(graphics=0)

# Save the modified document
doc.save('output.pdf')

# Close the document
doc.close()

In [9]:
#Get all the text
doc = pymupdf.open('demo.pdf')

text = ""
for page in doc:
    text += page.get_text()


In [10]:
text

'Background: Easy to use and secure redaction tool “RE-DACT” which allows\nredaction/masking/anonymization on various input formats based on a gradational scale defined\nby the user and providing customized output. Over a time, model will learn and have the ability to\ngenerate realistic synthetic data in any sought format. Description: The proposed solution is a\nnatural language processing (machine learning) based redaction tool. The tool will redact or\nobfuscate from original data leaving the output structurally/logically the same but stripped of key\nidentifiers and other content which may in any way allow the identity, actual data, markers or\nissues in the input content to be revealed. The correlational logic may be appropriately\nobfuscated based on the degree of redaction. This will have an easy to use GUI and will be\navailable for use on online and offline systems. The degree of the redaction will be up to the user-\nthe higher the degree set by the user, the more the degree

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("token-classification", model="output_mdeberta_base/checkpoint-320000" ,  aggregation_strategy="first")

2024-09-27 22:08:39.377155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 22:08:39.461922: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 22:08:39.476517: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 22:08:39.574645: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 22:08:40.689626: W tensorflow/compiler/tf2

In [2]:
text = "1234567891"
pipe(text )

/home/arnesh/.local/lib/python3.12/site-packages/transformers/pipelines/token_classification.py:392: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


[{'entity_group': 'ZIPCODE',
  'score': 0.9792691,
  'word': '1234567891',
  'start': 0,
  'end': 10}]

In [3]:
id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id


In [4]:
labels = list(id2label.values())
print(labels)

['O', 'B-CITY', 'I-CITY', 'B-FIRSTNAME', 'I-FIRSTNAME', 'B-ZIPCODE', 'I-ZIPCODE', 'B-DATE', 'I-DATE', 'B-CREDITCARDISSUER', 'I-CREDITCARDISSUER', 'B-CREDITCARDNUMBER', 'I-CREDITCARDNUMBER', 'B-CREDITCARDCVV', 'B-USERNAME', 'I-USERNAME', 'B-JOBTYPE', 'B-PREFIX', 'I-PREFIX', 'B-LASTNAME', 'B-EMAIL', 'I-EMAIL', 'B-IP', 'I-IP', 'B-PHONEIMEI', 'I-PHONEIMEI', 'B-MAC', 'I-MAC', 'B-DOB', 'I-DOB', 'B-NEARBYGPSCOORDINATE', 'I-NEARBYGPSCOORDINATE', 'B-BUILDINGNUMBER', 'I-BUILDINGNUMBER', 'B-IPV4', 'I-IPV4', 'B-BITCOINADDRESS', 'I-BITCOINADDRESS', 'B-SSN', 'I-SSN', 'B-PHONENUMBER', 'I-PHONENUMBER', 'B-SEX', 'B-SECONDARYADDRESS', 'I-SECONDARYADDRESS', 'B-ACCOUNTNUMBER', 'I-ACCOUNTNUMBER', 'B-ACCOUNTNAME', 'I-ACCOUNTNAME', 'B-STREET', 'I-STREET', 'B-MASKEDNUMBER', 'I-MASKEDNUMBER', 'B-COMPANYNAME', 'I-COMPANYNAME', 'B-CURRENCYSYMBOL', 'B-IBAN', 'I-IBAN', 'I-AMOUNT', 'B-IPV6', 'I-IPV6', 'B-MIDDLENAME', 'I-MIDDLENAME', 'B-COUNTY', 'I-COUNTY', 'B-AGE', 'B-PASSWORD', 'I-PASSWORD', 'B-STATE', 'I-STATE', 

In [6]:
entity_dict = {
    "NAME": ['B-FIRSTNAME', 'I-FIRSTNAME', 'B-LASTNAME', 'I-LASTNAME', 'B-MIDDLENAME', 'I-MIDDLENAME'],
    "CITY": ['B-CITY', 'I-CITY'],
    "ZIPCODE": ['B-ZIPCODE', 'I-ZIPCODE'],
    "DATE": ['B-DATE', 'I-DATE'],
    "CREDITCARD": ['B-CREDITCARDISSUER', 'I-CREDITCARDISSUER', 'B-CREDITCARDNUMBER', 'I-CREDITCARDNUMBER', 'B-CREDITCARDCVV', 'I-CREDITCARDCVV'],
    "USERNAME": ['B-USERNAME', 'I-USERNAME'],
    "JOB": ['B-JOBTYPE', 'I-JOBTYPE', 'B-JOBTITLE', 'I-JOBTITLE', 'B-JOBAREA', 'I-JOBAREA'],
    "PREFIX": ['B-PREFIX', 'I-PREFIX'],
    "EMAIL": ['B-EMAIL', 'I-EMAIL'],
    "IP": ['B-IP', 'I-IP', 'B-IPV4', 'I-IPV4', 'B-IPV6', 'I-IPV6'],
    "PHONE": ['B-PHONENUMBER', 'I-PHONENUMBER', 'B-PHONEIMEI', 'I-PHONEIMEI'],
    "MAC": ['B-MAC', 'I-MAC'],
    "DOB": ['B-DOB', 'I-DOB'],
    "GPS": ['B-NEARBYGPSCOORDINATE', 'I-NEARBYGPSCOORDINATE'],
    "ADDRESS": ['B-BUILDINGNUMBER', 'I-BUILDINGNUMBER', 'B-STREET', 'I-STREET', 'B-SECONDARYADDRESS', 'I-SECONDARYADDRESS'],
    "BITCOIN": ['B-BITCOINADDRESS', 'I-BITCOINADDRESS'],
    "SSN": ['B-SSN', 'I-SSN'],
    "SEX": ['B-SEX', 'I-SEX', 'B-GENDER', 'I-GENDER'],
    "ACCOUNT": ['B-ACCOUNTNUMBER', 'I-ACCOUNTNUMBER', 'B-ACCOUNTNAME', 'I-ACCOUNTNAME'],
    "CURRENCY": ['B-CURRENCYSYMBOL', 'I-CURRENCYSYMBOL', 'B-CURRENCY', 'I-CURRENCY', 'B-AMOUNT', 'I-AMOUNT', 'B-CURRENCYCODE', 'I-CURRENCYCODE', 'B-CURRENCYNAME', 'I-CURRENCYNAME'],
    "IBAN": ['B-IBAN', 'I-IBAN'],
    "BIC": ['B-BIC', 'I-BIC'],
    "URL": ['B-URL', 'I-URL'],
    "PASSWORD": ['B-PASSWORD', 'I-PASSWORD'],
    "STATE": ['B-STATE', 'I-STATE'],
    "COUNTY": ['B-COUNTY', 'I-COUNTY'],
    "AGE": ['B-AGE', 'I-AGE'],
    "TIME": ['B-TIME', 'I-TIME'],
    "HEIGHT": ['B-HEIGHT', 'I-HEIGHT'],
    "EYECOLOR": ['B-EYECOLOR', 'I-EYECOLOR'],
    "COMPANYNAME": ['B-COMPANYNAME', 'I-COMPANYNAME'],
    "ETHEREUM": ['B-ETHEREUMADDRESS', 'I-ETHEREUMADDRESS'],
    "LITECOIN": ['B-LITECOINADDRESS', 'I-LITECOINADDRESS'],
    "VEHICLE": ['B-VEHICLEVRM', 'I-VEHICLEVRM', 'B-VEHICLEVIN', 'I-VEHICLEVIN'],
    "ORDINALDIRECTION": ['B-ORDINALDIRECTION'],
    "USERAGENT": ['B-USERAGENT', 'I-USERAGENT'],
    "PIN": ['B-PIN', 'I-PIN'],
    "MASKEDNUMBER": ['B-MASKEDNUMBER', 'I-MASKEDNUMBER'],
}


In [7]:
#Mask in output
to_show_entities = ["NAME"]
to_show_entities_actual = []
for entity in to_show_entities:
    to_show_entities_actual += entity_dict[entity]
from transformers import pipeline


text = "Arnesh Batra"
out = pipe(text)
print(out)  
for ent_gr  in out:
    entity_gr = "I-" + ent_gr['entity_group']

    if entity_gr in to_show_entities_actual:
        text = text.replace(ent_gr['word'], "XXXXX")

print(text)

    

[{'entity_group': 'FIRSTNAME', 'score': 0.99994016, 'word': 'Arnesh', 'start': 0, 'end': 6}, {'entity_group': 'LASTNAME', 'score': 0.9999752, 'word': 'Batra', 'start': 6, 'end': 12}]
XXXXX XXXXX
